In [ ]:
from sentence_transformers import SentenceTransformer
import requests
import duckdb
import numpy as np
from duckvdb import DuckVDB
import json


In [2]:
#pokedata: list[dict] = requests.get("https://raw.githubusercontent.com/Purukitto/pokemon-data.json/refs/heads/master/pokedex.json").json()

In [3]:
#save json
# with open("pokemon.json", "w") as f:
#     json.dump(pokedata, f)


In [4]:
pokedata = json.load(open("pokemon.json"))

In [5]:
first_10_pokemon: list[dict] = pokedata[:10]



In [ ]:
first_10_pokemon

In [7]:
class MiniLMEMbedder:
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.vector_dim = self.model.get_sentence_embedding_dimension()

    def __call__(self, x: str) -> np.array:
        return self.model.encode(x)


embedder = MiniLMEMbedder()


In [ ]:
embedder.vector_dim

In [9]:
dvdb = DuckVDB(db_path="pokemon.duckdb", embedding_function=embedder)

In [10]:
metadatas: list[dict] = [json.dumps(pokemon) for pokemon in first_10_pokemon] #[{"name": pokemon["name"], "description": pokemon["description"]} for pokemon in first_10_pokemon]
embeddings: list[np.array] = [embedder(pokemon["description"]) for pokemon in first_10_pokemon]

dvdb.write(metadatas, embeddings)

In [ ]:
dvdb.query("water", k=10, filters={"base.Attack": ("<=", 49)})

In [12]:
# embedding_fn = EmbeddingFunction.from_method(<app-name>, <fn-name>) / EmbeddingFunction.from_class(<function-name>) / EmbeddingFunction()
# if no embedding_fn provided, need to validate on record entry
# mdb = DB("<db-name>", embedding_fn="<>") # if embedding_fn is class(validated), or name, check modal if function exists (check esisting mapping)



In [13]:
embedding_fn = MiniLMEMbedder()
from typing import Any, Optional, List
import modal
app = modal.App("modal-vector-db")
image = modal.Image.debian_slim()\
    .pip_install(
        "duckdb", 
        "pandas", 
        "pyarrow", 
        "numpy"
    )
vol = modal.Volume.from_name("modal-vector-db-volume", create_if_missing=True)
MOUNT_PATH = "/db"
@app.cls(volumes={MOUNT_PATH: vol}, image=image)
class WrapperClass:
    name: str = modal.parameter()

    @modal.enter()
    def enter(self):
        if embedding_fn is callable:
            embedding_model = embedding_fn
        else:
            embedding_model = modal.Function.lookup(embedding_fn)
        self.db: DuckVDB = DuckVDB(db_path=f"{MOUNT_PATH}/{self.name}.duckdb", embedding_function=embedding_model)

    @modal.method()
    def insert(self, metadatas: list[dict[str, Any]], embeddings: List[np.array]):
        self.db.write(metadatas, embeddings)

    @modal.method()
    def query(self, query: str, k: int = 10, filters: Optional[dict[str, Any]] = None):
        return self.db.query(query, k, filters)

In [14]:
remote_db = WrapperClass(name="pokemon_db")
with app.run():
    remote_db.insert.remote(metadatas, embeddings)
    remote_db.query.remote("water", k=10, filters={"base.Attack": ("<=", 49)})


[Thread-6 (thread_inner)] 2025-02-01T15:02:55-0800 Loop attempt for _run_app.<locals>.heartbeat failed
Traceback (most recent call last):
  File "/opt/anaconda3/envs/modal-vectordb-env/lib/python3.11/site-packages/grpclib/client.py", line 205, in send_request
    release_stream = await stream.send_request(
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/modal-vectordb-env/lib/python3.11/site-packages/grpclib/protocol.py", line 367, in send_request
    await self.connection.write_ready.wait()
  File "/opt/anaconda3/envs/modal-vectordb-env/lib/python3.11/asyncio/locks.py", line 213, in wait
    await fut
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/modal-vectordb-env/lib/python3.11/site-packages/modal/_utils/grpc_utils.py", line 178, in retry_transient_errors
    return await fn(*args, metadata=metadata, timeout=timeout)
           ^^^^^^^^